In [1]:
from typing import Dict
import csv

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm
from adamp import AdamP

import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import os
import random
import time

In [2]:
l5kit.__version__

'1.1.0'

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(42)

## Config

In [20]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'model_params': {
        'model_architecture': 'resnet18',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "model_multi_resnet18_64_ras_224",
        'lr': 1e-4,
        'weight_path': "models/resnet18_Vlad/cont_1e5_bs_128"
    },

    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
        },

    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 64,
        'shuffle': True,
        'num_workers': 16
    },

    'val_data_loader': {
        'key': 'scenes/validate.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 16
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 12,
        'shuffle': False,
        'num_workers': 16
    },

    'train_params': {
        'max_num_steps': 25001,
        'checkpoint_every_n_steps': 1000,
        'image_coords': False
    }
}

## Loss

In [6]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
import numpy as np

import torch
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability
    
    #use image_coords
    if cfg['train_params']['image_coords']:
        error = error / 4

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)


## Prepare and load validation data

In [7]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "prediction_dataset"
dm = LocalDataManager(None)
rasterizer = build_rasterizer(cfg, dm)

In [8]:
# ===== GENERATE AND LOAD CHOPPED DATASET
num_frames_to_chop = 100
eval_cfg = cfg["val_data_loader"]
eval_base_path = create_chopped_dataset(dm.require(eval_cfg["key"]), cfg["raster_params"]["filter_agents_threshold"], 
                              num_frames_to_chop, cfg["model_params"]["future_num_frames"], MIN_FUTURE_STEPS)

extracting GT: 100%|██████████| 94694/94694 [06:06<00:00, 258.03it/s]


In [9]:
eval_zarr_path = str(Path(eval_base_path) / Path(dm.require(eval_cfg["key"])).name)
eval_mask_path = str(Path(eval_base_path) / "mask.npz")
eval_gt_path = str(Path(eval_base_path) / "gt.csv")

eval_zarr = ChunkedDataset(eval_zarr_path).open()
eval_mask = np.load(eval_mask_path)["arr_0"]
# ===== INIT DATASET AND LOAD MASK
eval_dataset = AgentDataset(cfg, eval_zarr, rasterizer, agents_mask=eval_mask)
eval_dataloader = DataLoader(eval_dataset, shuffle=eval_cfg["shuffle"], batch_size=eval_cfg["batch_size"], 
                             num_workers=eval_cfg["num_workers"])
print(eval_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16220    |  1622000   | 125423254  |    11733321   |      45.06      |        100.00        |        77.33         |        10.00         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


## Model

In [10]:
class LyftMultiModel(nn.Module):

    def __init__(self, cfg: Dict, num_modes=3):
        super().__init__()

        # TODO: support other than resnet18?
        architecture = cfg["model_params"]["model_architecture"]
        backbone = eval(architecture)(pretrained=True, progress=True)
        self.backbone = backbone

        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )

        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        if architecture == "resnet50":
            backbone_out_features = 2048
        else:
            backbone_out_features = 512

        # X, Y coords for the future positions (output shape: Bx50x2)
        self.future_len = cfg["model_params"]["future_num_frames"]
        num_targets = 2 * self.future_len

        # You can add more layers here.
        self.head = nn.Sequential(
            #nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.num_preds = num_targets * num_modes
        self.num_modes = num_modes

        self.logit = nn.Linear(4096, out_features=self.num_preds + num_modes)

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.head(x)
        x = self.logit(x)

        # pred (bs)x(modes)x(time)x(2D coords)
        # confidences (bs)x(modes)
        bs, _ = x.shape
        pred, confidences = torch.split(x, self.num_preds, dim=1)
        pred = pred.view(bs, self.num_modes, self.future_len, 2)
        assert confidences.shape == (bs, self.num_modes)
        confidences = torch.softmax(confidences, dim=1)
        return pred, confidences

In [11]:
def forward(data, model, device, criterion = pytorch_neg_multi_log_likelihood_batch):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    
    matrix = data["world_to_image"].to(device)
    centroid = data["centroid"].to(device)[:,None,:].to(torch.float)
    rs = cfg["raster_params"]["raster_size"]
    ec = cfg["raster_params"]["ego_center"]
    
    bs,tl,_ = targets.shape
    assert tl == cfg["model_params"]["future_num_frames"]
    
    # Forward pass
    preds, confidences = model(inputs)    
    
    if cfg['train_params']['image_coords']:
        targets = targets + centroid
        targets = torch.cat([targets,torch.ones((bs,tl,1)).to(device)], dim=2)
        targets = torch.matmul(matrix.to(torch.float), targets.transpose(1,2))
        targets = targets.transpose(1,2)[:,:,:2]
        bias = torch.tensor([rs[0] * ec[0], rs[1] * ec[1]])[None, None, :].to(device)
        targets = targets - bias
        

    loss = criterion(targets, preds, confidences, target_availabilities)
    
    if cfg['train_params']['image_coords']:
        matrix_inv = torch.inverse(matrix)
        preds = preds + bias[:,None,:,:]
        preds = torch.cat([preds,torch.ones((bs,3,tl,1)).to(device)], dim=3)
        preds = torch.stack([torch.matmul(matrix_inv.to(torch.float), preds[:,i].transpose(1,2)) 
                            for i in range(3)], dim=1)
        preds = preds.transpose(2,3)[:,:,:,:2]
        preds = preds - centroid[:,None,:,:]
    
    return loss, preds, confidences

In [12]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = LyftMultiModel(cfg)
# weight_path = cfg["model_params"]["weight_path"]

# #read model
# model.load_state_dict(torch.load(weight_path))
# model.to(device)

## Prediction

In [13]:
def inference(model, eval_dataloader):
    
    model.eval()
    torch.set_grad_enabled(False)
    
    # store information for evaluation
    future_coords_offsets_pd = []
    timestamps = []
    confidences_list = []
    agent_ids = []

    progress_bar = tqdm(eval_dataloader)
    for data in progress_bar:
        _, preds, confidences = forward(data, model, device)

        #fix for the new environment
        preds = preds.cpu().numpy()
        world_from_agents = data["world_from_agent"].numpy()
        centroids = data["centroid"].numpy()
        coords_offset = []

        for idx in range(len(preds)):
            for mode in range(3):
                preds[idx, mode, :, :] = transform_points(preds[idx, mode, :, :], world_from_agents[idx]) - centroids[idx][:2]

        future_coords_offsets_pd.append(preds.copy())
        confidences_list.append(confidences.cpu().numpy().copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy())
        
    write_pred_csv('output/preds',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
               confs = np.concatenate(confidences_list)
              )  
    
    metrics = compute_metrics_csv(eval_gt_path, 'output/preds', [neg_multi_log_likelihood, time_displace])
    
    return metrics['neg_multi_log_likelihood']    

In [21]:
start_iter = 20000
end_iter = cfg['train_params']['max_num_steps']
step = cfg['train_params']['checkpoint_every_n_steps']
progress_bar = tqdm(range(start_iter,end_iter,step))
iterations = []
losses = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weight_path = cfg["model_params"]["weight_path"]

#keep track of validation metric
out_file = os.path.join(weight_path,f"validation_metrics_{start_iter}_{end_iter}_{step}.csv") 
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)
writer.writerow(['iterations', 'losses'])
of_connection.close()

for i in progress_bar:
    
    model = LyftMultiModel(cfg)
    pretrained_path = os.path.join(weight_path, f"model_{i}.pth")

    #read model
    model.load_state_dict(torch.load(pretrained_path))
    model.to(device)
    
    loss = inference(model, eval_dataloader)
    
    iterations.append(i)
    losses.append(loss)
    
    print(f"validation loss of model_{i} is {loss}")
    
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([i, loss])

results = pd.DataFrame({'iterations': iterations, 'losses': losses})
results.to_csv(out_file, index = False)
print(results)

  5%|▌         | 152/2960 [00:36<08:23,  5.58it/s]


 10%|█         | 303/2960 [01:07<09:15,  4.79it/s]


 15%|█▌        | 454/2960 [01:39<15:25,  2.71it/s]


 20%|██        | 605/2960 [02:09<07:08,  5.50it/s]


 26%|██▌       | 756/2960 [02:39<06:11,  5.93it/s]


...

 86%|████████▌ | 2537/2960 [08:40<01:21,  5.21it/s]


 91%|█████████ | 2685/2960 [09:11<01:05,  4.22it/s]


 96%|█████████▌| 2833/2960 [09:41<00:22,  5.65it/s]


100%|██████████| 6/6 [1:06:21<00:00, 663.56s/it]

validation loss of model_25000 is 16.069377525934208
   iterations     losses
0       20000  16.097150
1       21000  16.116591
2       22000  16.095791
3       23000  16.102268
4       24000  16.098357
5       25000  16.069378


In [15]:
#inference(model, eval_dataloader)

In [16]:
# # ==== EVAL LOOP
# model.eval()
# torch.set_grad_enabled(False)

# # store information for evaluation
# future_coords_offsets_pd = []
# timestamps = []
# confidences_list = []
# agent_ids = []

# progress_bar = tqdm(eval_dataloader)
# for data in progress_bar:
#     #inputs = data['image'].to(device)
#     #preds, confidences  = model(inputs)
#     _, preds, confidences = forward(data, model, device)
    
#     #fix for the new environment
#     preds = preds.cpu().numpy()
#     world_from_agents = data["world_from_agent"].numpy()
#     centroids = data["centroid"].numpy()
#     coords_offset = []

# #     for agent_coords, world_from_agent, centroid in zip(preds, world_from_agents, centroids):
# #         for i in range(3):
# #             agent_coords[i, :, :] = transform_points(agent_coords[i, :, :], world_from_agent) - centroid[:2]
# #         coords_offset.append(agent_coords)

#     # convert into world coordinates and compute offsets
#     for idx in range(len(preds)):
#         for mode in range(3):
#             preds[idx, mode, :, :] = transform_points(preds[idx, mode, :, :], world_from_agents[idx]) - centroids[idx][:2]
    
#     #future_coords_offsets_pd.append(np.stack(coords_offset))
#     #future_coords_offsets_pd.append(preds.cpu().numpy().copy())
#     future_coords_offsets_pd.append(preds.copy())
#     confidences_list.append(confidences.cpu().numpy().copy())
#     timestamps.append(data["timestamp"].numpy().copy())
#     agent_ids.append(data["track_id"].numpy().copy())

In [17]:
# pred_path = f'output/pred_resnet34_64_82000_105000'
# write_pred_csv(pred_path,
#                timestamps=np.concatenate(timestamps),
#                track_ids=np.concatenate(agent_ids),
#                coords=np.concatenate(future_coords_offsets_pd),
#                confs = np.concatenate(confidences_list)
#               )

In [18]:
# metrics = compute_metrics_csv(eval_gt_path, pred_path, [neg_multi_log_likelihood, time_displace])
# for metric_name, metric_mean in metrics.items():
#     print(metric_name, metric_mean)

In [19]:
#metrics